In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.SupportResistanceStrategy import SMAScanner

In [ ]:
with open('cfg.txt') as f:

    token = f.read()

In [ ]:
import requests

url = "https://api.orats.io/datav2/hist/dailies"

querystring = {"token":token,"ticker":"AAPL","tradeDate":"2019-11-29"}

response = requests.request("GET", url, params=querystring)

print(response.text)


In [ ]:
import json
with open('backtest.json', 'r') as file:
    backtest = json.load(file)

headers = {
    "Authorization": token
}

response = requests.post(
    "https://api.orats.io/backtest/submit",
    headers=headers,
    data=backtest
)

In [ ]:
print("Status Code:", response.status_code)
print("Response Body:", response.text)

In [ ]:
with open('files/watchlist.txt') as f:
    watchlist = f.read()

# df_raw = yf.download(watchlist.split(' '), auto_adjust=False, period='2y')

In [ ]:
scanner = SMAScanner(sma_periods=[50, 200], return_window=20)
output = scanner.run_scan(df_raw) # type: ignore
output

In [3]:
with open('files/watchlist.txt') as f:
    watchlist = f.read()

df_raw = yf.download(watchlist.split(' '), auto_adjust=False, interval='60m', period='3mo')

[*********************100%***********************]  80 of 80 completed


In [4]:
df = df_raw.stack(level=[0,1], future_stack=True).reset_index()
df.columns=['Timestamp', 'Variable', 'Ticker', 'Value']
df

,Timestamp,Variable,Ticker,Value
0,2025-04-14 13:30:00+00:00,Adj Close,AAPL,2.047500e+02
1,2025-04-14 13:30:00+00:00,Adj Close,ACN,2.915500e+02
2,2025-04-14 13:30:00+00:00,Adj Close,ADBE,3.566000e+02
3,2025-04-14 13:30:00+00:00,Adj Close,ADI,1.817500e+02
4,2025-04-14 13:30:00+00:00,Adj Close,AMAT,1.468827e+02
...,...,...,...,...
203035,2025-07-11 19:30:00+00:00,Volume,XLK,8.077370e+05
203036,2025-07-11 19:30:00+00:00,Volume,XLV,1.680563e+06
203037,2025-07-11 19:30:00+00:00,Volume,XOM,2.699434e+06
203038,2025-07-11 19:30:00+00:00,Volume,XOP,3.363220e+05


In [5]:
# assert set(df.columns) >= {'Timestamp', 'Variable', 'Ticker', 'Value'}
df_wide = df.pivot(index=['Timestamp', 'Ticker'], columns='Variable', values='Value').reset_index()
results = {}

window = 33
buffer = 0.01
for ticker, df_ticker in df_wide.groupby('Ticker'):
    df_ticker = df_ticker.sort_values('Timestamp').copy()
    df_ticker.set_index("Timestamp", inplace=True)
    if 'Open' not in df_ticker or 'Close' not in df_ticker:
        continue
    df_ticker = df_ticker.dropna(subset=['Open', 'Close'])
    body_top = df_ticker[['Open', 'Close']].max(axis=1)
    body_bottom = df_ticker[['Open', 'Close']].min(axis=1)
    pivot_high = (body_top == body_top.rolling(window, center=True).max())
    pivot_low = (body_bottom == body_bottom.rolling(window, center=True).min())
    df_ticker['pivot_high'] = pivot_high
    df_ticker['pivot_low'] = pivot_low
    resistance = df_ticker.loc[df_ticker['pivot_high'], ['Open', 'Close']].max(axis=1).tolist()
    support = df_ticker.loc[df_ticker['pivot_low'], ['Open', 'Close']].min(axis=1).tolist()

    # Group close levels
    def group_levels(levels):
        levels = sorted(levels)
        grouped = []
        for lvl in levels:
            if not grouped or abs(lvl - grouped[-1]) / grouped[-1] > buffer:
                grouped.append(lvl)
        return grouped

    results[ticker] = {
        "support": group_levels(support),
        "resistance": group_levels(resistance)
    }


In [13]:
df_wide.groupby('Ticker').resample('1d', on='Timestamp')['Close'].last().dropna()

Ticker  Timestamp                
AAPL    2025-04-14 00:00:00+00:00    202.589996
        2025-04-15 00:00:00+00:00    202.130005
        2025-04-16 00:00:00+00:00    194.300003
        2025-04-17 00:00:00+00:00    196.869995
        2025-04-21 00:00:00+00:00    193.089996
                                        ...    
XYZ     2025-07-07 00:00:00+00:00     69.419998
        2025-07-08 00:00:00+00:00     67.839996
        2025-07-09 00:00:00+00:00     69.010002
        2025-07-10 00:00:00+00:00     68.750000
        2025-07-11 00:00:00+00:00     65.080002
Name: Close, Length: 4880, dtype: float64